In [20]:
pip install mstarpy 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
import json
with open('transaction_detail.json') as f:
    transactions = json.load(f)


In [22]:
with open('transaction_detail.json') as f:
    transactions = json.load(f)
for i, trxn in enumerate(transactions):
    print(f"Transaction {i}: {type(trxn)}")


Transaction 0: <class 'dict'>


In [23]:
for i, trxn in enumerate(transactions):
    if isinstance(trxn, str): 
        print(f"Transaction {i} is a string, trying to parse")
        if trxn.strip():
            try:
                transactions[i] = json.loads(trxn) 
            except json.JSONDecodeError:
                print(f"Transaction {i} is not valid JSON, skipping")
                continue
        else:
            print(f"Transaction {i} is an empty string, skipping")
    elif isinstance(trxn, dict):
        print(f"Transaction {i} is already a dictionary")
    else:
        print(f"Unexpected data format in transaction {i}, skipping")


Transaction 0 is already a dictionary


In [24]:
pip install numpy-financial

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [25]:
import json
import datetime
from collections import defaultdict
import numpy as np
import numpy_financial as npf

# Load the transaction data
with open('transaction_detail.json') as f:
    data = json.load(f)

portfolio_value = 0
portfolio_gain = 0

cashflows = []
dates = []

for summary in data[0]['data']:
    for scheme in summary['dtSummary']:
        closing_balance = float(scheme['closingBalance'])
        nav = float(scheme['nav'])
        cost_value = float(scheme['costValue'])

        current_value = closing_balance * nav
        acquisition_cost = cost_value

        portfolio_value += current_value
        portfolio_gain += (current_value - acquisition_cost)

for transaction in data[0]['data']:
    for trxn in transaction['dtTransaction']:
        if trxn['trxnDesc'] == 'Purchase':  
            amount = float(trxn['trxnAmount'])
            trxn_date = datetime.datetime.strptime(trxn['trxnDate'], '%d-%b-%Y')
            cashflows.append(-amount) 
            dates.append(trxn_date)
cashflows.append(portfolio_value)
dates.append(datetime.datetime.now())

portfolio_xirr = npf.irr(cashflows) * 100  

print(f"Total Portfolio Value: {portfolio_value:.2f}")
print(f"Total Portfolio Gain: {portfolio_gain:.2f}")
print(f"Portfolio XIRR: {portfolio_xirr:.2f}%")


Total Portfolio Value: 4568788.70
Total Portfolio Gain: 1150385.79
Portfolio XIRR: 1.30%
